<a href="https://colab.research.google.com/github/CarlosARossi/dataScienceCoder/blob/main/ProyectoDS_ParteIII_%2BRossi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estructurando un Proyecto de DS (parte III)


### Consigna

Crearás un notebook que complemente el trabajo realizado en los siguientes apartados: 
1. Elegir un método de feature selection para reducir la dimensionalidad del dataset.
2. Elegir un algoritmo de regresión o clasificación para entrenar con los datos elegidos.
3. Cálculo de métricas para validar el modelo.
4. Generar conclusiones con base en los resultados obtenidos.






### 1. Importación de librerias y datasets

In [161]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import numpy as np

Importo el dataset generado en el notebook [ProyectoDS_ParteII_+Rossi.ipynb](https://github.com/CarlosARossi/dataScienceCoder/blob/main/ProyectoDS_ParteII_%2BRossi.ipynb)

In [162]:
url = 'https://raw.githubusercontent.com/CarlosARossi/dataScienceCoder/main/Datasets/Salary_Data_2022_REV15_USD_Months.csv'
salaries = pd.read_csv(url)
salaries

,ID,Timestamp,Age.Range,Years.of.Experience,Industry,Job.Title,Company.Name,Education,Location,Country,...,Open.To.Discuss.Salary,How.many.months.Maternity.or.Paternity.does.your.company.offer.,Gender,X,Diverse.Identity..Optional.,Description,Exchange.rate.to.USD,SalaryUSD,Salary.Period,Monthly.SalaryUSD
0,1,12/8/2021 3:05,18-21,3.0,GAMING,STREAMER,TWITCH,None,LOS ANGELES CA,US,...,No,as much as i want,Male,NaN,NaN,TWITCH,1.00000,490000.000000,Annual,40833.333333
1,2,12/9/2021 15:49,30-33,5.0,HEALTHCARE,AESTHETIC NURSE PRACTITIONER/BUSINESS OWNER,AESTHETIC CLINIC,Master's Degree,WINNIPEG,CANADA,...,Yes,NaN,Male,NaN,NaN,AESTHETIC CLINIC,1.28848,380293.058488,Annual,31691.088207
2,3,12/8/2021 2:16,18-21,1.0,ONLINE ARCADE,OWNER,RUST CLASH,High School/GED,SAN DIEGO,US,...,Yes,NaN,Male,NaN,NaN,RUST CLASH,1.00000,480000.000000,Annual,40000.000000
3,4,12/8/2021 5:15,18-21,2.0,CRYPTO,HEAD OF MARKETING,REDACTED,Some College,"TAMPA, FL",US,...,Yes,Not sure,Male,NaN,NaN,REDACTED,1.00000,480000.000000,Annual,40000.000000
4,5,12/8/2021 14:16,34-37,13.0,WINDOW COVERINGS,OWNER,CAROLINA BLIND CRAFTERS,Bachelor's Degree,"CHARLOTTE, NC",US,...,No,None,Female,NaN,NaN,CAROLINA BLIND CRAFTERS,1.00000,480000.000000,Annual,40000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31757,31758,12/17/2021 1:49,30-33,5.0,ECOMMERCE,DESIGN MANAGER,MIS,Bachelor's Degree,MUMBAI,INDIA,...,No,NaN,Male,NaN,NaN,MIS,75.87400,0.092258,Monthly,0.092258
31758,31759,12/23/2021 4:12,22-25,5.0,HOTEL/RESTAURANT,MANAGER/CHEF,CASA BAROLO,Bachelor's Degree,"CLUJ-NAPOCA, CLUJ",ROMANIA,...,Yes,24,Female,NaN,NaN,CASA BAROLO,4.39980,1.363698,Monthly,1.363698
31759,31760,12/7/2021 19:40,58+,NaN,SPORTS,ANALYST,ESPN,None,SAN JOSE,US,...,No,0,Female,NaN,They,ESPN,1.00000,1.000000,Annual,0.083333
31760,32569,1/7/2022 12:40,26-29,4.0,MATERIAL HANDLING,SOLUTIONS CONSULTANT,DEMATIC,Bachelor's Degree,"GRAND RAPIDS, MI",US,...,Yes,NaN,Male,NaN,Hispanic,DEMATIC,1.00000,110000.000000,Annual,9166.666667


### 2. Análisis inicial

In [163]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31762 entries, 0 to 31761
Data columns (total 28 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   ID                                                               31762 non-null  int64  
 1   Timestamp                                                        31762 non-null  object 
 2   Age.Range                                                        31742 non-null  object 
 3   Years.of.Experience                                              31260 non-null  float64
 4   Industry                                                         31558 non-null  object 
 5   Job.Title                                                        31758 non-null  object 
 6   Company.Name                                                     31760 non-null  object 
 7   Education                               

Procedo a analizar del dataset la cantidad de instancias únicas de las caracteristicas consideradas como variables nominales:

In [164]:
# Contar la cantidad de instancias únicas de las caracteristicas considerads como variables nominales

print("Cantidad de Generos:", len(salaries['Gender'].unique()))
print("Cantidad de Paises:", len(salaries['Country'].unique()))
print("Cantidad de Rangos de edad:", len(salaries['Age.Range'].unique()))
print("Cantidad de Industrias:", len(salaries['Industry'].unique()))
print("Cantidad de Puestos de trabajo:", len(salaries['Job.Title'].unique()))


Cantidad de Generos: 3
Cantidad de Paises: 55
Cantidad de Rangos de edad: 12
Cantidad de Industrias: 5618
Cantidad de Puestos de trabajo: 12592


Debido a la alta cantidad de instancias únicas que tienen las variables nominales elegidas, principalmente **Industry** y **Job.Title**, puedo encontrarme con problemas de alta cardinalidad. Para poder evitar esto, procederé a aplicar una técnica de reducción manual basada en la Ley de Pareto.

### 3. Reducción de Instancias



In [165]:
# Crear una copia del dataset original
salaries_reduced = salaries.copy()

# Elimino las columnas que no me interesan
columns_to_keep = ['ID', 'Age.Range', 'Years.of.Experience', 'Industry', 'Job.Title', 'Education', 'Country', 'Gender', 'Monthly.SalaryUSD']
salaries_reduced = salaries_reduced[columns_to_keep]
salaries_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31762 entries, 0 to 31761
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   31762 non-null  int64  
 1   Age.Range            31742 non-null  object 
 2   Years.of.Experience  31260 non-null  float64
 3   Industry             31558 non-null  object 
 4   Job.Title            31758 non-null  object 
 5   Education            31762 non-null  object 
 6   Country              31762 non-null  object 
 7   Gender               31762 non-null  object 
 8   Monthly.SalaryUSD    31762 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 2.2+ MB


#### 3.1. Rango de edades [Age.Range]

Verifico que no haya quedado ninguna instancia con NaN

In [166]:
# Contar la cantidad de NaN en la columna seleccionada
nan_count = salaries_reduced['Age.Range'].isna().sum()

# Imprimir el resultado
print("Cantidad de NaN en la columna '{}': {}".format('Age.Range', nan_count))

Cantidad de NaN en la columna 'Age.Range': 20


In [167]:
# Reemplazar los NaN por 'OTHERS'
salaries_reduced['Age.Range'].fillna('OTHERS', inplace=True)

# Verifico el resultado
nan_count = salaries_reduced['Age.Range'].isna().sum()
print("Cantidad de NaN en la columna '{}': {}".format('Age.Range', nan_count))

Cantidad de NaN en la columna 'Age.Range': 0


#### 3.2. Años de experiencia [Years.of.Experience]

Verifico que no haya quedado ninguna instancia con NaN

In [168]:
# Contar la cantidad de NaN en la columna seleccionada
nan_count = salaries_reduced['Years.of.Experience'].isna().sum()

# Imprimir el resultado
print("Cantidad de NaN en la columna '{}': {}".format('Years.of.Experience', nan_count))

Cantidad de NaN en la columna 'Years.of.Experience': 502


In [169]:
# Reemplazar los NaN por el valor medio de 'Years.of.Experience'
# Calcular el valor medio de la columna 'A'
mean_value = salaries_reduced['Years.of.Experience'].mean()
salaries_reduced['Years.of.Experience'].fillna(mean_value, inplace=True)

# Verifico el resultado
nan_count = salaries_reduced['Years.of.Experience'].isna().sum()
print("Cantidad de NaN en la columna '{}': {}".format('Years.of.Experience', nan_count))

Cantidad de NaN en la columna 'Years.of.Experience': 0


#### 3.3. Industrias [Industry]

In [170]:
# Calcular la frecuencia de las categorías en Industry
industry_counts = salaries_reduced['Industry'].value_counts()

In [171]:
# Calcular el porcentaje acumulado
industry_cumsum = industry_counts.cumsum() / industry_counts.sum()

In [172]:
# Calcular el umbral para agrupar las categorías menos frecuentes
threshold = industry_cumsum[industry_cumsum > 0.5].index[0]

# Agrupar las categorías menos frecuentes en 'others'
salaries_reduced.loc[salaries_reduced['Industry'].isin(industry_counts[industry_counts < industry_counts[threshold]].index), 'Industry'] = 'OTHERS'

# Verificar los resultados
print(salaries_reduced['Industry'].value_counts())

print("Cantidad de Industrias:", len(salaries_reduced['Industry'].unique()))

OTHERS                15653
TECH                   3354
HEALTHCARE             1628
FINANCE                1087
EDUCATION               845
CONSULTING              740
RETAIL                  730
MARKETING               703
ACCOUNTING              634
INSURANCE               571
BANKING                 556
ADVERTISING             377
FINANCIAL SERVICES      288
LEGAL                   285
HIGHER EDUCATION        282
GOVERNMENT              275
HEALTH CARE             256
SALES                   237
REAL ESTATE             231
CONSTRUCTION            213
HOSPTECHALTECHY         208
MANUFACTURING           207
ENGINEERING             207
MEDIA                   203
AUTOMOTIVE              201
LAW                     194
FINTECH                 177
ENTERTAINMENT           167
MEDICAL                 163
PUBLIC ACCOUNTING       162
BIOTECH                 155
HUMAN RESOURCES         153
AEROSPACE               149
FASHION                 138
SOCIAL WORK             129
Name: Industry, dtyp

Verifico que no haya quedado ninguna instancia con NaN

In [173]:
# Contar la cantidad de NaN en la columna seleccionada
nan_count = salaries_reduced['Industry'].isna().sum()

# Imprimir el resultado
print("Cantidad de NaN en la columna '{}': {}".format('Industry', nan_count))

Cantidad de NaN en la columna 'Industry': 204


In [174]:
# Reemplazar los NaN por 'OTHERS'
salaries_reduced['Industry'].fillna('OTHERS', inplace=True)

# Verifico el resultado
nan_count = salaries_reduced['Industry'].isna().sum()
print("Cantidad de NaN en la columna '{}': {}".format('Industry', nan_count))

Cantidad de NaN en la columna 'Industry': 0


#### 3.4. Puestos de Trabajo [Job.Title]

In [175]:
# Calcular la frecuencia de las categorías en Job.Title
job_title_counts = salaries_reduced['Job.Title'].value_counts()


In [176]:
# Calcular el porcentaje acumulado
job_title_cumsum = job_title_counts.cumsum() / job_title_counts.sum()

In [177]:
# Calcular el umbral para agrupar las categorías menos frecuentes
threshold = job_title_cumsum[job_title_cumsum > 0.5].index[0]

# Agrupar las categorías menos frecuentes en 'others'
salaries_reduced.loc[salaries_reduced['Job.Title'].isin(job_title_counts[job_title_counts < job_title_counts[threshold]].index), 'Job.Title'] = 'OTHERS'

# Verificar los resultados
print(salaries_reduced['Job.Title'].value_counts())
print("Cantidad de Puestos de trabajo:", len(salaries_reduced['Job.Title'].unique()))

OTHERS                                 15315
SOFTWARE ENGINEER                        451
ANALYST                                  426
PROJECT MANAGER                          377
ASSOCIATE                                306
                                       ...  
CONTENT SPECIALIST                         6
MANAGEMENT TRAINEE                         6
MARKETING COMMUNICATIONS SPECIALIST        6
DIGITAL PRODUCER                           6
GRADUATE ASSISTANT                         6
Name: Job.Title, Length: 697, dtype: int64
Cantidad de Puestos de trabajo: 698


Verifico que no haya quedado ninguna instancia con NaN

In [178]:
# Contar la cantidad de NaN en la columna seleccionada
nan_count = salaries_reduced['Job.Title'].isna().sum()

# Imprimir el resultado
print("Cantidad de NaN en la columna '{}': {}".format('Job.Title', nan_count))

Cantidad de NaN en la columna 'Job.Title': 4


In [179]:
# Reemplazar los NaN por 'OTHERS'
salaries_reduced['Job.Title'].fillna('OTHERS', inplace=True)

# Verifico el resultado
nan_count = salaries_reduced['Job.Title'].isna().sum()
print("Cantidad de NaN en la columna '{}': {}".format('Job.Title', nan_count))

Cantidad de NaN en la columna 'Job.Title': 0


#### 3.5. País [Country]

In [180]:
# Calcular la frecuencia de las categorías en Country
country_counts = salaries_reduced['Country'].value_counts()

In [181]:
# Calcular el porcentaje acumulado
country_cumsum = country_counts.cumsum() / country_counts.sum()

In [182]:
# Calcular el umbral para agrupar las categorías menos frecuentes
threshold = country_cumsum[country_cumsum > 0.995].index[0]

# Agrupar las categorías menos frecuentes en 'others'
salaries_reduced.loc[salaries_reduced['Country'].isin(country_counts[country_counts < country_counts[threshold]].index), 'Country'] = 'OTHERS'

# Verificar los resultados
print(salaries_reduced['Country'].value_counts())
print("Cantidad de Países:", len(salaries_reduced['Country'].unique()))

US           29325
CANADA        1563
UK             591
OTHERS         145
GERMANY         59
SINGAPORE       33
AUSTRALIA       24
FRANCE          22
Name: Country, dtype: int64
Cantidad de Países: 8


Verifico que no haya quedado ninguna instancia con NaN

In [183]:
# Contar la cantidad de NaN en la columna seleccionada
nan_count = salaries_reduced['Country'].isna().sum()

# Imprimir el resultado
print("Cantidad de NaN en la columna '{}': {}".format('Country', nan_count))

Cantidad de NaN en la columna 'Country': 0


#### 3.5. Conclusión

In [184]:
# Contar la cantidad de instancias únicas de las caracteristicas considerads como variables nominales

print("Cantidad de Rango de Edades Antes:", len(salaries['Age.Range'].unique()))
print("Cantidad de Rango de Edades Ahora:", len(salaries_reduced['Age.Range'].unique()))
print("----------------------------------------")
print("Cantidad de Años de Experiencia Antes:", len(salaries['Years.of.Experience'].unique()))
print("Cantidad de Años de Experiencia Ahora:", len(salaries_reduced['Years.of.Experience'].unique()))
print("----------------------------------------")
print("Cantidad de Industrias Antes:", len(salaries['Industry'].unique()))
print("Cantidad de Industrias Ahora:", len(salaries_reduced['Industry'].unique()))
print("----------------------------------------")
print("Cantidad de Puestos de trabajo Antes:", len(salaries['Job.Title'].unique()))
print("Cantidad de Puestos de trabajo Ahora:", len(salaries_reduced['Job.Title'].unique()))
print("----------------------------------------")
print("Cantidad de Paises Antes:", len(salaries['Country'].unique()))
print("Cantidad de Paises Ahora:", len(salaries_reduced['Country'].unique()))
print("----------------------------------------")
print("Cantidad de Generos Antes:", len(salaries['Gender'].unique()))
print("Cantidad de Generos Ahora:", len(salaries_reduced['Gender'].unique()))

Cantidad de Rango de Edades Antes: 12
Cantidad de Rango de Edades Ahora: 12
----------------------------------------
Cantidad de Años de Experiencia Antes: 20
Cantidad de Años de Experiencia Ahora: 20
----------------------------------------
Cantidad de Industrias Antes: 5618
Cantidad de Industrias Ahora: 35
----------------------------------------
Cantidad de Puestos de trabajo Antes: 12592
Cantidad de Puestos de trabajo Ahora: 697
----------------------------------------
Cantidad de Paises Antes: 55
Cantidad de Paises Ahora: 8
----------------------------------------
Cantidad de Generos Antes: 3
Cantidad de Generos Ahora: 3


### 4. Feature Engineering


#### 4.1. One-Hot Encoding

Se aplicará One-Hot Encoding a las siguientes características:

* Country
* Age.Range
* Industry
* Job.Title
* Gender

Las cuales fueron elegidas debido a su naturaleza nominal y la cantidad moderada de instancias únicas en cada una de ellas. Esta técnica convierte las variables categóricas en columnas separadas con valores binarios. 

In [185]:
nominal_variables = ['Country', 'Age.Range', 'Industry', 'Job.Title', 'Gender']

# Aplicar One-Hot Encoding
salaries_one_hot = pd.get_dummies(salaries_reduced, columns=nominal_variables)

# Verificar los resultados
# print(salaries_encoded.head())
salaries_one_hot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31762 entries, 0 to 31761
Columns: 759 entries, ID to Gender_Male
dtypes: float64(2), int64(1), object(1), uint8(755)
memory usage: 23.8+ MB


#### 4.2. Ordinal Encoding

Se aplicará Ordinal Encoding a la siguiente caracteristica:

* Education  

Se eligió este método porque la variable Education tiene un orden implícito, como por ejemplo "High School" < "Bachelor's Degree" < "Master's Degree", y el Ordinal Encoding captura esta jerarquía de manera adecuada.

In [186]:
# Analiso cuales son las instancias unicas dentro de Education
salaries_one_hot['Education'].unique()

array(['None', "Master's Degree", 'High School/GED', 'Some College',
       "Bachelor's Degree", 'Doctorate Degree', 'Trade/Vocational',
       "Associate's Degree"], dtype=object)

In [187]:
from sklearn.preprocessing import OrdinalEncoder

# Crear una copia del dataset anterior
salaries_encoded = salaries_one_hot.copy()

# Definir el orden de los tipos de estudios
education_order = ['None', 'High School/GED', 'Trade/Vocational', 'Some College', "Associate's Degree", "Bachelor's Degree", "Master's Degree", 'Doctorate Degree']

# Crear el objeto OrdinalEncoder con el orden específico
ordinal_encoder = OrdinalEncoder(categories=[education_order])

# Aplicar el encoding ordinal a la columna 'Education'
salaries_encoded['Education'] = ordinal_encoder.fit_transform(salaries_encoded[['Education']])

# Verificar los resultados
salaries_encoded

,ID,Years.of.Experience,Education,Monthly.SalaryUSD,Country_AUSTRALIA,Country_CANADA,Country_FRANCE,Country_GERMANY,Country_OTHERS,Country_SINGAPORE,...,Job.Title_VICE PRESIDENT,Job.Title_VIDEO EDITOR,Job.Title_VIDEO PRODUCER,Job.Title_VISUAL DESIGNER,Job.Title_VP,Job.Title_WAITRESS,Job.Title_WEB DEVELOPER,Gender_Female,Gender_LGBTQ+,Gender_Male
0,1,3.000000,0.0,40833.333333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,5.000000,6.0,31691.088207,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,1.000000,1.0,40000.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,2.000000,3.0,40000.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,13.000000,5.0,40000.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31757,31758,5.000000,5.0,0.092258,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
31758,31759,5.000000,5.0,1.363698,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
31759,31760,4.329431,0.0,0.083333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
31760,32569,4.000000,5.0,9166.666667,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 5. Algoritmo de clasificación

#### 5.1. Random Forest



In [188]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.metrics import mean_squared_error, r2_score

In [189]:
X = salaries_encoded.drop("Monthly.SalaryUSD", axis=1)
y = salaries_encoded["Monthly.SalaryUSD"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()  
model.fit(X_train, y_train)

RandomForestRegressor()

In [190]:
y_pred = model.predict(X_test)

In [191]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Error cuadrático medio:", mse)
print("Coeficiente de determinación (R^2):", r2)

Error cuadrático medio: 81414.54688888103
Coeficiente de determinación (R^2): 0.9937174674659185


#### 5.2. Conclusión

Con base en los resultados obtenidos, podemos concluir lo siguiente:

1. El error cuadrático medio (MSE) obtenido fue de 77642.36535474448. Este valor indica la magnitud promedio de los errores al predecir los salarios mensuales. Un MSE más bajo indica un mejor ajuste del modelo a los datos. En este caso, el valor obtenido sugiere que el modelo de Random Forest tiene un buen rendimiento en la predicción de los salarios.

2. El coeficiente de determinación (R^2) obtenido fue de 0.9940085561486943. Este valor indica la proporción de la variabilidad en la variable objetivo (salarios mensuales) que puede explicarse por el modelo. Un valor de R^2 más cercano a 1 indica un mejor ajuste del modelo a los datos. En este caso, el valor obtenido sugiere que el modelo explica aproximadamente el 99.4% de la variabilidad en los salarios mensuales, lo cual es muy alto.

En conclusión, el modelo de Random Forest aplicado a los datos seleccionados muestra un rendimiento muy bueno, con un bajo error cuadrático medio y un alto coeficiente de determinación. Esto indica que el modelo es capaz de realizar predicciones precisas sobre los salarios mensuales en función de las características seleccionadas, como la edad, años de experiencia, industria, puesto de trabajo, educación, país y género.